### Libraries 

In [ ]:
import pandas as pd 
from datetime import datetime, timedelta, timezone
import os

### CSV File Reader

In [ ]:
def load_site_configuration(csv_file_path): 
    try:
        # Read the CSV file
        sites_df = pd.read_csv(csv_file_path)
        
        # Filter out rows where Project Name is empty/null
        sites_df = sites_df.dropna(subset=['Project Name'])
        sites_df = sites_df[sites_df['Project Name'].str.strip() != '']
        
        # Get unique site names
        site_names = sites_df['Project Name'].unique().tolist()
        
        print(f"Loaded {len(site_names)} sites from configuration file:")
        for i, site in enumerate(site_names, 1):
            print(f"  {i}. {site}")
        
        return site_names, sites_df
        
    except Exception as e:
        print(f"Error loading site configuration: {e}")
        return [], pd.DataFrame()


### Generate metadata

In [ ]:
# Generate metadata for inverter module state signals for a specific site
def generate_inv_module_metadata(site_name):
    try:
        print(f"Generating metadata for site: {site_name}")
        
        # Search for inverter module signals
        inv_mod_ac_power = spy.search({
            'Path': f'GPM >> {site_name} >> Inverter module',
            'Name': 'Active Power',
            'Type': 'Signal'
        }).sort_values('Asset').reset_index(drop=True)
        
        inv_mod_ac_power = inv_mod_ac_power[
            (inv_mod_ac_power["Name"] == 'ACTIVE POWER') & 
            (inv_mod_ac_power["Asset"].str.contains("Module"))
        ].reset_index(drop=True)
        
        # look for POWERELECTRONICS HEx CURRENT FAULT
        inv_mod_fault_code = spy.search({
            'Path': f'GPM >> {site_name} >> Inverter module',
            'Name': 'POWERELECTRONICS HEx CURRENT FAULT',
            'Type': 'Signal'
        }).sort_values(['Asset']).reset_index(drop=True)
        
        inv_mod_comms_status = spy.search({
            'Path': f'GPM >> {site_name} >> Inverter module',
            'Name': 'COMS STATUS',
            'Type': 'Signal'
        }).sort_values('Asset').reset_index(drop=True)
        
        plant_irrad = spy.search({
            'Path': f'GPM >> {site_name} >> Customized element',
            'Name': 'Average Filtered POA_sub (HC)',
            'Type': 'Signal'
        })
        
        if plant_irrad.empty:
            print(f"Warning: No irradiance data found for {site_name}")
            return None
        
        # Generate metadata for each module
        metadata_list = []
        for i in range(len(inv_mod_ac_power.index)):
            formula = '''$no_data = $ap.isNotValid()
            $sufficient_irrad = ($irrad>20).merge(0min).removeShorterThan(20min)
            $inv_no_power = ($ap==0).merge(0min).removeShorterThan(20min)
            $inv_online = ($ap>0).intersect($sufficient_irrad)
            $comms_out = ($comstat == 255).merge(0min).removeShorterThan(20min)
            $fault_active = ($faultcode !=0).merge(0min).removeShorterThan(20min)
            $inv_good_comms = ($comstat == 0).merge(0min).removeShorterThan(20min)
            $fault_inactive = ($faultcode ==0).merge(0min).removeShorterThan(20min)
            $der = $ap.derivative('h')
            $freeze = ($der == 0)
            $comms_freeze = $freeze.intersect($sufficient_irrad).intersect($inv_online).intersect($fault_inactive).removeShorterThan(20min)
            $inv_offline = ($inv_no_power).intersect($sufficient_irrad)
            $inv_comms_out = ($inv_offline).intersect($comms_out).removeShorterThan(20min)
            $inv_off_fault = $inv_offline.intersect($inv_good_comms).intersect($fault_active).removeShorterThan(20min)
            $inv_off_no_fault = $inv_offline.intersect($inv_good_comms).intersect($fault_inactive).removeShorterThan(20min)
            "0".splice("OFFLINE WITH FAULT",$inv_off_fault).splice("OFFLINE WITHOUT FAULT",$inv_off_no_fault).splice("COMMS OUTAGE",$inv_comms_out).splice("COMMS FREEZE",$comms_freeze).splice("MISSING DATA",$no_data)'''
            
            metadata_df = {
                'Path': inv_mod_ac_power['Path'].iloc[i],
                'Asset': inv_mod_ac_power['Asset'].iloc[i],
                'Name': 'Inv Module State',
                'Type': 'Signal',              
                'Formula': formula,
                'Formula Parameters': {
                    '$irrad': plant_irrad.loc[0,'ID'],
                    '$ap': inv_mod_ac_power.loc[i,'ID'],
                    '$comstat': inv_mod_comms_status.loc[i,'ID'],
                    '$faultcode': inv_mod_fault_code.loc[i,'ID']
                }
            }
            metadata_list.append(pd.DataFrame([metadata_df]))
        
        all_metadata = pd.concat(metadata_list, ignore_index=True)
        print(f"Generated metadata for {len(all_metadata)} modules at {site_name}")
        
        return all_metadata
        
    except Exception as e:
        print(f"Error generating metadata for {site_name}: {e}")
        return None

In [ ]:
# Analyze fault codes and return fault periods with start/end dates for each module
def analyze_fault_periods(site_name, filtered_data, fault_codes):

    if fault_codes is None or (filtered_data is None or filtered_data.empty):
        print(f"No fault code data available for analysis at {site_name}")
        return []
    
    fault_periods = []
    
    # Analyze each module
    for column in fault_codes.columns:
        module_faults = fault_codes[column].dropna()
        
        if module_faults.empty:
            continue
            
        # Find fault periods (non-zero fault codes)
        fault_active = module_faults[module_faults != 0]
        
        if fault_active.empty:
            continue
            
        # Group consecutive fault periods
        current_fault = None
        start_time = None
        prev_time = None
        
        for timestamp, fault_code in fault_active.items():
            if current_fault is None:
                # Start of a new fault period
                current_fault = fault_code
                start_time = timestamp
                prev_time = timestamp
            elif fault_code != current_fault or (timestamp - prev_time).total_seconds() > 1800:  # 30 min gap
                # Fault code changed or significant time gap, end previous period
                fault_periods.append({
                    'Site': site_name,
                    'Module': column,
                    'Start': start_time,
                    'End': prev_time,
                    'Fault_Code': current_fault,
                    'Duration_Hours': round((prev_time - start_time).total_seconds() / 3600, 2)
                })
                current_fault = fault_code
                start_time = timestamp
            
            prev_time = timestamp
        
        # Don't forget the last fault period
        if current_fault is not None:
            fault_periods.append({
                'Site': site_name,
                'Module': column,
                'Start': start_time,
                'End': prev_time,
                'Fault_Code': current_fault,
                'Duration_Hours': round((prev_time - start_time).total_seconds() / 3600, 2)
            })
    
    return fault_periods


In [ ]:
# Run fault analysis for all sites and generate comprehensive report
def run_multi_site_analysis(csv_file_path, start_date, end_date, output_file='fault_analysis_report.csv', grid='5min'):
 
    print("="*80)
    print("MULTI-SITE INVERTER MODULE FAULT ANALYSIS")
    print("="*80)
    
    # Load site configuration
    site_names, sites_df = load_site_configuration(csv_file_path)
    
    if not site_names:
        print("No sites found in configuration file")
        return None
    
    # Initialize results storage
    all_fault_periods = []
    site_summary = []
    
    # Process each site
    for site_name in site_names:
        try:
            fault_periods = analyze_single_site(site_name, start_date, end_date, grid) 
             
            if fault_periods:
                all_fault_periods.extend(fault_periods)
                
                # Create site summary
                site_summary.append({
                    'Site': site_name,
                    'Total_Fault_Periods': len(fault_periods),
                    'Total_Modules_Affected': len(set([fp['Module'] for fp in fault_periods])),
                    'Total_Fault_Hours': sum([fp['Duration_Hours'] for fp in fault_periods]),
                    'Analysis_Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                })
            else:
                site_summary.append({
                    'Site': site_name,
                    'Total_Fault_Periods': 0,
                    'Total_Modules_Affected': 0,
                    'Total_Fault_Hours': 0,
                    'Analysis_Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                })
                
        except Exception as e:
            print(f"Failed to analyze {site_name}: {e}")
            site_summary.append({
                'Site': site_name,
                'Total_Fault_Periods': 'ERROR',
                'Total_Modules_Affected': 'ERROR',
                'Total_Fault_Hours': 'ERROR',
                'Analysis_Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })
    
    # Convert results to DataFrames
    if all_fault_periods:
        fault_df = pd.DataFrame(all_fault_periods)
        # Format datetime columns
        fault_df['Start'] = pd.to_datetime(fault_df['Start'])
        fault_df['End'] = pd.to_datetime(fault_df['End'])
        fault_df['Start_Formatted'] = fault_df['Start'].dt.strftime('%Y-%m-%d %H:%M:%S')
        fault_df['End_Formatted'] = fault_df['End'].dt.strftime('%Y-%m-%d %H:%M:%S')
        
        # Reorder columns for better readability
        fault_df = fault_df[['Site', 'Module', 'Start_Formatted', 'End_Formatted', 
                            'Fault_Code', 'Duration_Hours']]
    else:
        fault_df = pd.DataFrame()
    
    summary_df = pd.DataFrame(site_summary)
    
    # Save results to CSV
    try:
        if not fault_df.empty:
            fault_df.to_csv(output_file, index=False)
            print(f"\nFault analysis report saved to: {output_file}")
        
        summary_file = output_file.replace('.csv', '_summary.csv')
        summary_df.to_csv(summary_file, index=False)
        print(f"Site summary saved to: {summary_file}")
        
    except Exception as e:
        print(f"Error saving results: {e}")
    
    # Display summary
    print("\n" + "="*80)
    print("ANALYSIS SUMMARY")
    print("="*80)
    
    total_sites = len(site_names)
    sites_with_faults = len([s for s in site_summary if s['Total_Fault_Periods'] > 0])
    total_fault_periods = sum([s['Total_Fault_Periods'] for s in site_summary if isinstance(s['Total_Fault_Periods'], int)])
    
    print(f"Total Sites Analyzed: {total_sites}")
    print(f"Sites with Faults: {sites_with_faults}")
    print(f"Total Fault Periods: {total_fault_periods}")
    
    if not fault_df.empty:
        print(f"Total Fault Hours: {fault_df['Duration_Hours'].sum():.2f}")
        print(f"Average Fault Duration: {fault_df['Duration_Hours'].mean():.2f} hours")
    
    return fault_df, summary_df


### Data Filtering Function 

In [ ]:
def filter_ims_data(site_name, start_date, end_date, grid='5min', target_states=None):
    try:
        # Default target states
        if target_states is None:
            target_states = ['OFFLINE WITH FAULT', 'OFFLINE W/O FAULT', 'COMMS OUTAGE']
        
        # Search for Inv Module State signals
        mod_state = spy.search({
            'Path': f'GPM >> {site_name}',
            'Name': 'Inv Module State',
            'Type': 'Signal'
        }).sort_values('Asset').reset_index(drop=True)
        
        if mod_state.empty:
            print(f"No Inv Module State signals found for {site_name}")
            return None, None
        
        # Pull the data for the found signals
        mod_data = spy.pull(
            mod_state, 
            start=start_date, 
            end=end_date, 
            grid=grid
        )
        
        # Filter the data to isolate modules with target states
        filtered_data = mod_data[mod_data.isin(target_states)]
        
        # Remove rows where all values are NaN (no target states found)
        filtered_data = filtered_data.dropna(how='all')
        
        # Get fault codes for offline modules
        fault_codes = None
        try:
            # Search for fault code signals
            fault_code_signals = spy.search({
                'Path': f'GPM >> {site_name} >> Inverter module',
                'Name': 'POWERELECTRONICS HEx CURRENT FAULT',
                'Type': 'Signal'
            }).sort_values(['Asset']).reset_index(drop=True)
            
            if not fault_code_signals.empty:
                # Pull fault code data
                fault_codes = spy.pull(
                    fault_code_signals,
                    start=start_date,
                    end=end_date,
                    grid=grid
                )
                
                print(f"Found fault codes for {len(fault_code_signals)} modules at {site_name}")
            else:
                print(f"No fault code signals found for {site_name}")
                
        except Exception as e:
            print(f"Could not retrieve fault codes for {site_name}: {e}")
            fault_codes = None
        
        return filtered_data, fault_codes
        
    except Exception as e:
        print(f"Error filtering data for {site_name}: {e}")
        return None, None


### Fault Period Analysis 

In [ ]:
# Analyze fault codes and return fault periods with start/end dates for each module
def analyze_fault_periods(site_name, filtered_data, fault_codes):
    
    if fault_codes is None or (filtered_data is None or filtered_data.empty):
        print(f"No fault code data available for analysis at {site_name}")
        return []
    
    fault_periods = []
    
    # Analyze each module
    for column in fault_codes.columns:
        module_faults = fault_codes[column].dropna()
        
        if module_faults.empty:
            continue
            
        # Find fault periods (non-zero fault codes)
        fault_active = module_faults[module_faults != 0]
        
        if fault_active.empty:
            continue
            
        # Group consecutive fault periods
        current_fault = None
        start_time = None
        prev_time = None
        
        for timestamp, fault_code in fault_active.items():
            if current_fault is None:
                # Start of a new fault period
                current_fault = fault_code
                start_time = timestamp
                prev_time = timestamp
            elif fault_code != current_fault or (timestamp - prev_time).total_seconds() > 1800:  # 30 min gap
                # Fault code changed or significant time gap, end previous period
                fault_periods.append({
                    'Site': site_name,
                    'Module': column,
                    'Start': start_time,
                    'End': prev_time,
                    'Fault_Code': current_fault,
                    'Duration_Hours': round((prev_time - start_time).total_seconds() / 3600, 2)
                })
                current_fault = fault_code
                start_time = timestamp
            
            prev_time = timestamp
        
        # Don't forget the last fault period
        if current_fault is not None:
            fault_periods.append({
                'Site': site_name,
                'Module': column,
                'Start': start_time,
                'End': prev_time,
                'Fault_Code': current_fault,
                'Duration_Hours': round((prev_time - start_time).total_seconds() / 3600, 2)
            })
    
    return fault_periods

### Single Site analysis 

In [27]:
def analyze_single_site(site_name, start_date, end_date, grid='5min'):

    print(f"Analyzing {site_name}...")
    
    try:
        # Filter data
        filtered_data, fault_codes = filter_ims_data(site_name, start_date, end_date, grid)
        
        if fault_codes is not None:
            # Analyze fault periods
            fault_periods = analyze_fault_periods(site_name, filtered_data, fault_codes)
            
            if fault_periods:
                print(f"  Found {len(fault_periods)} fault periods")
                return fault_periods
            else:
                print(f"  No fault periods found")
                return []
        else:
            print(f"  Could not retrieve fault code data")
            return []
            
    except Exception as e:
        print(f"  Error analyzing {site_name}: {e}")
        return []

### All site analysis and report 

In [28]:
def run_multi_site_analysis(csv_file_path, start_date, end_date, output_file='fault_analysis_report.csv', grid='5min'):
    print("Starting Multi-Site Inverter Module Fault Analysis...")
    
    # Load site configuration
    site_names, sites_df = load_site_configuration(csv_file_path)
    
    if not site_names:
        print("No sites found in configuration file")
        return None
    
    # Initialize results storage
    all_fault_periods = []
    site_summary = []
    
    # Process each site
    for site_name in site_names:
        try:
            fault_periods = analyze_single_site(site_name, start_date, end_date, grid)
            
            if fault_periods:
                all_fault_periods.extend(fault_periods)
                
                # Create site summary
                site_summary.append({
                    'Site': site_name,
                    'Total_Fault_Periods': len(fault_periods),
                    'Total_Modules_Affected': len(set([fp['Module'] for fp in fault_periods])),
                    'Total_Fault_Hours': sum([fp['Duration_Hours'] for fp in fault_periods]),
                    'Analysis_Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                })
            else:
                site_summary.append({
                    'Site': site_name,
                    'Total_Fault_Periods': 0,
                    'Total_Modules_Affected': 0,
                    'Total_Fault_Hours': 0,
                    'Analysis_Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                })
                
        except Exception as e:
            print(f"  Failed to analyze {site_name}: {e}")
            site_summary.append({
                'Site': site_name,
                'Total_Fault_Periods': 'ERROR',
                'Total_Modules_Affected': 'ERROR',
                'Total_Fault_Hours': 'ERROR',
                'Analysis_Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })
    
    # Convert results to DataFrames
    if all_fault_periods:
        fault_df = pd.DataFrame(all_fault_periods)
        # Format datetime columns
        fault_df['Start'] = pd.to_datetime(fault_df['Start'])
        fault_df['End'] = pd.to_datetime(fault_df['End'])
        fault_df['Start_Formatted'] = fault_df['Start'].dt.strftime('%m/%d/%Y %I %p')
        fault_df['End_Formatted'] = fault_df['End'].dt.strftime('%m/%d/%Y %I %p')
        
        # Create the formatted fault period string
        fault_df['Fault_Period'] = (fault_df['Module'] + ': ' + 
                                   fault_df['Start_Formatted'] + ' - ' + 
                                   fault_df['End_Formatted'] + ', Fault=' + 
                                   fault_df['Fault_Code'].astype(str))
        
        # Reorder columns for better readability
        fault_df = fault_df[['Site', 'Module', 'Start_Formatted', 'End_Formatted', 
                            'Fault_Code', 'Duration_Hours', 'Fault_Period']]
    else:
        fault_df = pd.DataFrame()
    
    summary_df = pd.DataFrame(site_summary)
    
    # Save results to CSV
    try:
        if not fault_df.empty:
            fault_df.to_csv(output_file, index=False)
            print(f"Fault analysis report saved to: {output_file}")
        
        summary_file = output_file.replace('.csv', '_summary.csv')
        summary_df.to_csv(summary_file, index=False)
        print(f"Site summary saved to: {summary_file}")
        
    except Exception as e:
        print(f"Error saving results: {e}")
    
    # Display summary
    print("\nAnalysis Summary:")
    
    total_sites = len(site_names)
    sites_with_faults = len([s for s in site_summary if s['Total_Fault_Periods'] > 0])
    total_fault_periods = sum([s['Total_Fault_Periods'] for s in site_summary if isinstance(s['Total_Fault_Periods'], int)])
    
    print(f"Total Sites Analyzed: {total_sites}")
    print(f"Sites with Faults: {sites_with_faults}")
    print(f"Total Fault Periods: {total_fault_periods}")
    
    if not fault_df.empty:
        print(f"Total Fault Hours: {fault_df['Duration_Hours'].sum():.2f}")
        print(f"Average Fault Duration: {fault_df['Duration_Hours'].mean():.2f} hours")
    
    return fault_df, summary_df

### Run main()

In [29]:
def main():
    
    # Parameters
    CSV_FILE_PATH = 'PE(Sheet1).csv'  
    START_DATE = '2025-07-07'
    END_DATE = '2025-07-09'
    OUTPUT_FILE = 'inverter_fault_analysis_report.csv'
    GRID_RESOLUTION = '5min'
    
    print("Starting Multi-Site Inverter Module Fault Analysis...")
    print(f"Date Range: {START_DATE} to {END_DATE}")
    print(f"Grid Resolution: {GRID_RESOLUTION}")
    print(f"Output File: {OUTPUT_FILE}")
    
    # Run the analysis
    fault_df, summary_df = run_multi_site_analysis(
        csv_file_path=CSV_FILE_PATH,
        start_date=START_DATE,
        end_date=END_DATE,
        output_file=OUTPUT_FILE,
        grid=GRID_RESOLUTION
    )
    
    return fault_df, summary_df 

if __name__ == "__main__":
    results_df, summary_df = main()

Starting Multi-Site Inverter Module Fault Analysis...
Date Range: 2025-07-07 to 2025-07-09
Grid Resolution: 5min
Output File: inverter_fault_analysis_report.csv
Starting Multi-Site Inverter Module Fault Analysis...
Loaded 21 sites from configuration file:
  1. Aerojet- Camden
  2. Carey
  3. Carey B
  4. Fort Lupton
  5. Haywood
  6. Houston
  7. Kersey
  8. Mavericks
  9. Millington
  10. New Albany A
  11. New Albany B
  12. Platte
  13. Providence
  14. Ripley (Energize)
  15. Ripley (PS Lauderdale)
  16. Ripley (SEPI)
  17. Ripley (SR)
  18. Selmer 1
  19. Selmer 2
  20. South Loving
  21. Windsor
Analyzing Aerojet- Camden...
Error filtering data for Aerojet- Camden: name 'spy' is not defined
  Could not retrieve fault code data
Analyzing Carey...
Error filtering data for Carey: name 'spy' is not defined
  Could not retrieve fault code data
Analyzing Carey B...
Error filtering data for Carey B: name 'spy' is not defined
  Could not retrieve fault code data
Analyzing Fort Lupton...
